In [10]:
import pyodbc
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo


In [11]:
# Conexión con Access
conexion = pyodbc.connect(
    "DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    "DBQ=Datos/Gestion_Pedidos_Ejercicios.accdb;"
)


In [12]:
consulta1 = "select * from clientes"
consulta2 = "select * from pedidos"
df_clientes = pd.read_sql(consulta1, conexion)
df_pedidos = pd.read_sql(consulta2, conexion)
conexion.close()


In [13]:
df_clientes.head(10)


,CÓDIGO CLIENTE,EMPRESA,DIRECCIÓN,POBLACIÓN,TELÉFONO,RESPONSABLE
0,CT01,BELTRÁN E HIJOS,LAS FUENTES 78,MADRID,914456435,ANGEL MARTÍNEZ
1,CT02,LA MODERNA,LA PALOMA 123,OVIEDO,985323434,JUAN GARCÍA
2,CT03,EL ESPAÑOLITO,MOTORES 34,BARCELONA,934565343,ANA FERNÁNDEZ
3,CT04,EXPORTASA,VALLECAS 34,MADRID,913452378,ELVIRA GÓMEZ
4,CT06,CONFECCIONES AMPARO,LOS MOROS 23,GIJÓN,985754332,LUÍS ÁLVAREZ
5,CT07,LA CASA DEL JUGUETE,AMÉRICA 45,MADRID,912649987,ELÍAS PÉREZ
6,CT08,JUGUETERÍA SUÁREZ,PARIS 123,BARCELONA,933457866,JUAN GARCÍA
7,CT09,ALMACÉN POPULAR,LAS FUENTES 124,BILBAO,942347127,JOSÉ ÁLVAREZ
8,CT10,FERETERÍA EL CLAVO,PASEO DE ÁLAMOS 78,MADRID,914354866,MANUEL MENÉNDEZ
9,CT11,JUGUETES MARTÍNEZ,VIA LAYETANA 245,BARCELONA,936628554,FRANCISCO CUEVAS


In [14]:
df_pedidos.head()


,NÚMERO DE PEDIDO,CÓDIGO CLIENTE,FECHA DE PEDIDO,FORMA DE PAGO,DESCUENTO,ENVIADO
0,1,CT01,2000-03-11,CONTADO,0.02,True
1,3,CT23,2000-03-18,APLAZADO,0.06,False
2,5,CT25,2000-03-31,CONTADO,0.09,False
3,7,CT12,2000-04-12,CONTADO,0.07,False
4,8,CT01,2000-04-15,TARJETA,0.02,True


In [15]:
# Cruce de las tablas
df_merge = pd.merge(
    # Va a buscar a la tabla pedidos los el código cliente y se generan registros
    df_clientes,
    # Sólo se quiere que traiga la columna fecha de pedido de la tabla pedidos (de igual forma se especifica la columna en común)
    df_pedidos[['CÓDIGO CLIENTE', 'FECHA DE PEDIDO']],
    on='CÓDIGO CLIENTE',
    how='left'
).fillna('No hizo pedido')

df_merge


,CÓDIGO CLIENTE,EMPRESA,DIRECCIÓN,POBLACIÓN,TELÉFONO,RESPONSABLE,FECHA DE PEDIDO
0,CT01,BELTRÁN E HIJOS,LAS FUENTES 78,MADRID,914456435,ANGEL MARTÍNEZ,2000-03-11 00:00:00
1,CT01,BELTRÁN E HIJOS,LAS FUENTES 78,MADRID,914456435,ANGEL MARTÍNEZ,2000-04-15 00:00:00
2,CT01,BELTRÁN E HIJOS,LAS FUENTES 78,MADRID,914456435,ANGEL MARTÍNEZ,2002-08-18 00:00:00
3,CT01,BELTRÁN E HIJOS,LAS FUENTES 78,MADRID,914456435,ANGEL MARTÍNEZ,2001-08-02 00:00:00
4,CT01,BELTRÁN E HIJOS,LAS FUENTES 78,MADRID,914456435,ANGEL MARTÍNEZ,2002-09-17 00:00:00
...,...,...,...,...,...,...,...
63,CT36,JUGUETES EDUCATIVOS SANZ,ORENSE 89,MADRID,916872354,PEDRO IGLESIAS,No hizo pedido
64,CT37,ALMACENES FERNANDEZ,ANTÓN 67,TERUEL,978564025,MARIA ARDANZA,No hizo pedido
65,CT38,CONFECCIONES MÓNICA,MOTORES 67,BARCELONA,935681245,PEDRO SERRANO,No hizo pedido
66,CT39,FERRETERÍA LIMA,VALLECAS 45,MADRID,913532785,LUIS GARCÍA,No hizo pedido


In [16]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 0 to 67
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CÓDIGO CLIENTE   68 non-null     object
 1   EMPRESA          68 non-null     object
 2   DIRECCIÓN        68 non-null     object
 3   POBLACIÓN        68 non-null     object
 4   TELÉFONO         68 non-null     object
 5   RESPONSABLE      68 non-null     object
 6   FECHA DE PEDIDO  68 non-null     object
dtypes: object(7)
memory usage: 4.2+ KB


In [17]:
# Pegado de la tabla en Excel
wb = xw.Book('Reportes/Cruce.xlsx')
wb.app.screen_updating = False
wb.app.visible = False
wb.sheets["Hoja1"].clear_contents()
wb.sheets['Hoja1'].range(1, 1).value = df_merge.columns.tolist()
wb.sheets['Hoja1'].range(2, 1).value = df_merge.values
wb.save('Reportes/Cruce.xlsx')
wb.close()


In [18]:
# Formato tabla
libro = load_workbook('Reportes/Cruce.xlsx')
hoja = libro.get_sheet_by_name('Hoja1')
min_fila = hoja.min_row
max_fila = hoja.max_row
tab = Table(displayName="Table1", ref=f'A{min_fila}:G{max_fila}')
style = TableStyleInfo(
    name="TableStyleMedium9",
    showFirstColumn=False,
    showLastColumn=False,
    showRowStripes=True,
    showColumnStripes=True
)

tab.tableStyleInfo = style
hoja.add_table(tab)
libro.save('Reportes/Cruce.xlsx')


C:\Users\usuario\AppData\Local\Temp/ipykernel_15876/1179205605.py:3: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  hoja = libro.get_sheet_by_name('Hoja1')
